# Pytorch Distributed Training

See the "Pytorch distributed training" section of the [README](README.md) for setting up the Ray cluster before running this example.

For more information about Ray Train and the pytorch distributed training example, please check the original [getting-started-pytorch](https://docs.ray.io/en/latest/train/getting-started-pytorch.html) documentation.

## 1. Install requirements

In [1]:
%%bash
# pip install "ray[data,train,tune,serve]"==2.9.0
# pip install torch torchvision
# pip install IPython

Run a port forwarding to the Ray head service:
```bash
kubectl port-forward svc/raycluster-kuberay-head-svc 10001:10001 -n default
```

In [1]:
import os
import tempfile

import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Normalize, Compose

import ray
from ray.train.torch import TorchTrainer
import ray.train.torch

print(ray.__version__)

2.9.0


## 2. Connect to the Ray cluster

In [2]:
runtime_env = {
    "pip": ["torch", "torchvision", "IPython"],
}
ray.init(address="ray://localhost:10001", runtime_env=runtime_env)

Python version:,3.8.13
Ray version:,2.7.0
Dashboard:,http://10.244.0.79:8265


In [3]:
print(ray.cluster_resources())

{'object_store_memory': 6313651814.0, 'node:10.244.0.79': 1.0, 'node:__internal_head__': 1.0, 'node:10.244.0.78': 1.0, 'memory': 22000000000.0, 'GPU': 2.0, 'CPU': 8.0, 'accelerator_type:G': 2.0}


## 3. Define training

- `train_func` is the Python code that executes on each distributed training worker.

- `ScalingConfig` defines the number of distributed training workers and whether to use GPUs.

- `TorchTrainer` launches the distributed training job.

In [4]:
def train_func(config):
    # Model, Loss, Optimizer
    model = resnet18(num_classes=10)
    model.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )
    # model.to("cuda")  # This is done by `prepare_model`
    # [1] Prepare model.
    model = ray.train.torch.prepare_model(model)
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)

    # Data
    transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
    data_dir = os.path.join(tempfile.gettempdir(), "data")
    train_data = FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    # [2] Prepare dataloader.
    train_loader = ray.train.torch.prepare_data_loader(train_loader)

    # Training
    n_epochs = 4
    for epoch in range(n_epochs):
        for images, labels in train_loader:
            # This is done by `prepare_data_loader`!
            # images, labels = images.to("cuda"), labels.to("cuda")
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # [3] Report metrics and checkpoint.
        metrics = {"loss": loss.item(), "epoch": epoch}
        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            torch.save(
                model.module.state_dict(),
                os.path.join(temp_checkpoint_dir, "model.pt")
            )
            ray.train.report(
                metrics,
                checkpoint=ray.train.Checkpoint.from_directory(temp_checkpoint_dir),
            )
        if ray.train.get_context().get_world_rank() == 0:
            print(metrics)

In [5]:
# Configure scaling and resource requirements.
scaling_config = ray.train.ScalingConfig(num_workers=2, use_gpu=False)

In [6]:
# Distributed training job.
trainer = ray.train.torch.TorchTrainer(
    train_func,
    scaling_config=scaling_config,
    run_config=ray.train.RunConfig(
        storage_path="/home/ray/nfs",
        name="nfs",
    )
)

## 4. Launch training job

In [7]:
result = trainer.fit()

(TunerInternal pid=3003) [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


(TunerInternal pid=3003) 
(TunerInternal pid=3003) View detailed results here: /home/ray/nfs/nfs
(TunerInternal pid=3003) To visualize your results with TensorBoard, run: `tensorboard --logdir /home/ray/ray_results/nfs`


(TunerInternal pid=3003) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.
(TunerInternal pid=3003) GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(TrainTrainable pid=619, ip=10.244.0.78) GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.
(TorchTrainer pid=619, ip=10.244.0.78) GPUs are detected in your Ray cluster, but GPU training is not enabled for this trainer. To enable GPU training, make sure to set `use_gpu` to True in your scaling config.


(TunerInternal pid=3003) 
(TunerInternal pid=3003) Training started without custom configuration.


(TorchTrainer pid=619, ip=10.244.0.78) Starting distributed worker processes: ['675 (10.244.0.78)', '676 (10.244.0.78)']
(RayTrainWorker pid=675, ip=10.244.0.78) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=675, ip=10.244.0.78) Moving model to device: cpu
(RayTrainWorker pid=675, ip=10.244.0.78) Wrapping provided model in DistributedDataParallel.


...

(RayTrainWorker pid=676, ip=10.244.0.78) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/ray/nfs/nfs/TorchTrainer_bf0eb_00000_0_2024-01-12_01-27-03/checkpoint_000001)
(RayTrainWorker pid=675, ip=10.244.0.78) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/ray/nfs/nfs/TorchTrainer_bf0eb_00000_0_2024-01-12_01-27-03/checkpoint_000001)


(TunerInternal pid=3003) 
(TunerInternal pid=3003) Training finished iteration 3 at 2024-01-12 01:41:09. Total running time: 14min 5s
(TunerInternal pid=3003) ╭─────────────────────────────────────────╮
(TunerInternal pid=3003) │ Training result                         │
(TunerInternal pid=3003) ├─────────────────────────────────────────┤
(TunerInternal pid=3003) │ checkpoint_dir_name   checkpoint_000002 │
(TunerInternal pid=3003) │ time_this_iter_s               186.5469 │
(TunerInternal pid=3003) │ time_total_s                  564.97583 │
(TunerInternal pid=3003) │ training_iteration                    3 │
(TunerInternal pid=3003) │ epoch                                 2 │
(TunerInternal pid=3003) │ loss                            0.16846 │
(TunerInternal pid=3003) ╰─────────────────────────────────────────╯
(TunerInternal pid=3003) Training saved a checkpoint for iteration 3 at: (local)/home/ray/nfs/nfs/TorchTrainer_bf0eb_00000_0_2024-01-12_01-27-03/checkpoint_000002
(RayTrainWork

(RayTrainWorker pid=676, ip=10.244.0.78) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/ray/nfs/nfs/TorchTrainer_bf0eb_00000_0_2024-01-12_01-27-03/checkpoint_000002)
(RayTrainWorker pid=675, ip=10.244.0.78) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/ray/nfs/nfs/TorchTrainer_bf0eb_00000_0_2024-01-12_01-27-03/checkpoint_000002)


(TunerInternal pid=3003) 
(TunerInternal pid=3003) Training finished iteration 4 at 2024-01-12 01:44:15. Total running time: 17min 11s
(TunerInternal pid=3003) ╭─────────────────────────────────────────╮
(TunerInternal pid=3003) │ Training result                         │
(TunerInternal pid=3003) ├─────────────────────────────────────────┤
(TunerInternal pid=3003) │ checkpoint_dir_name   checkpoint_000003 │
(TunerInternal pid=3003) │ time_this_iter_s               186.0695 │
(TunerInternal pid=3003) │ time_total_s                  751.04532 │
(TunerInternal pid=3003) │ training_iteration                    4 │
(TunerInternal pid=3003) │ epoch                                 3 │
(TunerInternal pid=3003) │ loss                            0.16956 │
(TunerInternal pid=3003) ╰─────────────────────────────────────────╯
(TunerInternal pid=3003) Training saved a checkpoint for iteration 4 at: (local)/home/ray/nfs/nfs/TorchTrainer_bf0eb_00000_0_2024-01-12_01-27-03/checkpoint_000003
(RayTrainWor

(RayTrainWorker pid=676, ip=10.244.0.78) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/ray/nfs/nfs/TorchTrainer_bf0eb_00000_0_2024-01-12_01-27-03/checkpoint_000003)
(RayTrainWorker pid=675, ip=10.244.0.78) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/ray/nfs/nfs/TorchTrainer_bf0eb_00000_0_2024-01-12_01-27-03/checkpoint_000003)
